Librerías

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as grd
import csv
import numpy as np
from scipy.optimize import curve_fit

Carga de datos

In [ ]:
archivo = "archivo_1.data"
archivo2 = "archivo_2.data"

Lectura de archivos

In [ ]:
def read_data(path):
    with open(path, 'r') as file:
        data = [int(line.split(' ')[0]) for line in file]
    return data
  
datos = read_data(archivo)
datos2 = read_data(archivo2)

datosn = [d for d in datos if 80 <= d <= 20000]
datosn2 =[d for d in datos2 if 80 <= d <= 20000]

Ajuste

In [ ]:
def exponentialfit(x, A, T):
    return A * np.exp(-x / T)  

Ocurrencias

In [ ]:
valores_unicos, ocurrencias = np.unique(datosn, return_counts=True)
valores_unicos2 , ocurrencias2 = np.unique(datosn2, return_counts=True)

Contenedor de datos

In [ ]:
num_bins = 20
bins = np.linspace(0, 20000, num_bins + 1)
bin_indices = np.digitize(datosn, bins) - 1
bin_indices = np.clip(bin_indices, 0, num_bins - 1)
bin_indices2 = np.digitize(datosn2, bins) - 1
bin_indices2 = np.clip(bin_indices2, 0, num_bins - 1)

conteo_por_bin = np.bincount(bin_indices, minlength=num_bins)
suma_por_bin = np.bincount(bin_indices, weights=datosn, minlength=num_bins)
promedio_por_bin = np.divide(suma_por_bin, conteo_por_bin, where=conteo_por_bin > 0)

conteo_por_bin2 = np.bincount(bin_indices2, minlength=num_bins)
suma_por_bin2 = np.bincount(bin_indices2, weights=datosn2, minlength=num_bins)
promedio_por_bin2 = np.divide(suma_por_bin2,conteo_por_bin2, where=conteo_por_bin2 > 0)

Gráfica y análisis

In [ ]:
y = (bins[:-1] + bins[1:]) / 2
error_x = np.full(num_bins, (bins[1] - bins[0]) / 2)

mask = conteo_por_bin > 0
mask2 = conteo_por_bin2 > 0
guess = [10, 2000]
param, cov = curve_fit(exponentialfit, y[mask], conteo_por_bin[mask], p0=guess)
param2 , cov2 = curve_fit(exponentialfit,y[mask2], conteo_por_bin2[mask2], p0=guess)

residuales_1 = (conteo_por_bin - exponentialfit(y, param[0], param[1])) / np.sqrt(conteo_por_bin)
residuales_2 = (conteo_por_bin2 - exponentialfit(y, param2[0], param2[1])) / np.sqrt(conteo_por_bin2)

print(param, cov)

x_ = np.linspace(0, 20000, 100)
y_ = exponentialfit(x_, param[0], param[1])
y_2 = exponentialfit(x_, param2[0], param2[1])

fig = plt.figure(figsize=(10,5))
gs = grd.GridSpec(5, 3, height_ratios=[1, 1, 0.4, 1, 1], width_ratios=[1, 0.001, 2], hspace=0, wspace=0.5)

ax1 = plt.subplot(gs[:2, 0])
ax2 = plt.subplot(gs[-2:,0])

ax3 = plt.subplot(gs[0,2])
ax4 = plt.subplot(gs[1,2])
ax5 = plt.subplot(gs[3,2])
ax6 = plt.subplot(gs[4,2])

ax1.scatter(valores_unicos, ocurrencias,alpha=0.5,color='b', label='Raw Data')
ax1.set_ylabel("Toma de datos 1", color='b')
ax1.set_title("Tiempo vs Numero de decaimientos")

ax1.legend(loc='upper right')

ax2.scatter(valores_unicos2, ocurrencias2,alpha=0.5,color='b', label='Raw Data 2')
ax2.set_ylabel("Toma de datos 2", color='b')

ax2.legend(loc='upper right')

param_text = f"A = {param[0]:.2f}, T = {param[1]:.2f}"
ax3.plot(x_, y_, label=f"Ajuste exponencial,\n{param_text}", color="black")
ax3.errorbar(y, conteo_por_bin, xerr=error_x, fmt='o', color='r', capsize=5, label="Valor promedio con error en X")

ax3.set_ylabel("Toma\ndatos 1", color='r')
ax3.set_title("Tiempo vs Decaiminetos por bin")

ax3.legend(loc="upper right")

ax4.scatter(y, residuales_1, color='g', marker='x')
ax4.axhline(0, color='gray', linestyle='--')  # Línea base en y = 0
ax4.set_ylabel("Residuales\nNormalizados", rotation=90)
ax4.legend(loc="upper right")

param_text2 = f"A = {param2[0]:.2f}, T = {param2[1]:.2f}"
ax5.plot(x_, y_2 , label=f"Ajuste exponencial 2,\n{param_text2}", color='black')
ax5.errorbar(y, conteo_por_bin2, xerr=error_x, fmt="o", color="r", capsize=5, label="Valor promedio con error en X 2")

ax5.set_xlabel("Tiempo (ns)")
ax5.set_ylabel("Toma\ndatos 2", color='r')

ax5.legend(loc="upper right")

ax6.scatter(y, residuales_2, color='g', marker='x')
ax6.axhline(0, color='gray', linestyle='--')  # Línea base en y = 0
ax6.set_ylabel("Residuales\nNormalizados", rotation=90)
ax6.set_xlabel("Tiempo (ns)")
ax6.legend(loc="upper right")

plt.show()